In [1]:
#直接处理从sbib下载的cube文件，更新spice kernels， phocube，检查simulated image

#Dealing with Ceres cube files from sbib
#Directly generating the backplanes from web setting best spice kernels
import pysis
from pysis import isis
from pysis import CubeFile
from PIL import Image
import math
import matplotlib.pyplot as plt
import os
from os import path
import pysis_ext
import matplotlib.pyplot as plt


In [2]:
#help(pysis_ext.pds2isis)    #(input, output, log=logfile, ...)

In [3]:
#input_list and output_list
file_path = '/Users/zouxd/21DDAP-Li/02-OccatorCrater/Occator-data/'
out_path = '/Users/zouxd/21DDAP-Li/02-OccatorCrater/use-globalDTM/'
files = os.listdir(file_path) 
paths = [os.path.join(file_path,file) for file in files if file.endswith('.cub')]
print('%d cubes in the given path.' %(len(paths)))

out_paths = [os.path.join(out_path,file) for file in files if file.endswith('.cub')]

5969 cubes in the given path.


In [7]:
#spiceinit
for path in paths:
    print(path)
    isis.spiceinit(from_=path, web=True, cksmithed=True, spksmithed=True)

    


/Users/zouxd/21DDAP-Li/02-OccatorCrater/Occator-data/f-FC21B0064946_16131093537F1B.cub


ProcessError: Command '/Users/zouxd/Applications/isis/isis3comets/isis/bin/spiceinit' returned non-zero exit status 1

In [4]:
fi=files[3000:4000]
len(fi)
#print(files[5978])

1000

In [5]:
#phocube
for file in fi:
    if file.endswith('.cub'):
        print(file, end = "\r")
        #phocube from=*.cub to=*.cub dn=yes emission=no incidence=no localemission=yes localincidence=yes pixelresolution=yes
        #isis.phocube(from_=os.path.join(file_path,file), to=os.path.join(out_path,file),dn=True, emission=False, incidence=False, localemission=True, localincidence=True, sunilluminationmask=True)
        isis.phocube(from_=os.path.join(file_path,file), to=os.path.join(out_path,file), emission=False, incidence=False, localemission=True, localincidence=True,sunilluminationmask=True)


#LS model
def lsmodel(inc, emi, pha, exp1, exp2, exp3):
    f = math.exp(exp1*pha + exp2*pha**2+exp3*pha**3) * math.cos(inc*math.pi/180)/(math.cos(inc*math.pi/180)+math.cos(emi*math.pi/180))
    return f


#simulated image vs image
import numpy as np
for path in out_paths:
    image = CubeFile.open(path)
    data = image.get_image_array()
    #Save to png: 
    png1=path+'1.png'
    png2=path+'2.png'
    print(png1)
    img = Image.fromarray(data[0])
    img.save(png1)
    #inc,emi,pha 
    inc = data[3]
    emi = data[2]
    pha = data[1]
    ran = inc.shape
    sim = np.zeros((ran[0],ran[1]),dtype='float')
    for i in range(ran[0]):
        for j in range(ran[1]):
            #sim[i,j] = lsmodel(inc[i,j],emi[i,j], pha[i,j],-0.0327,0.00022842,-0.0000014225)
            sim[i,j] = math.cos(inc[i,j]*math.pi/180)
    img2=Image.fromarray(sim)
    imgplot = plt.imshow(sim)

#fx
for path in paths:
    # Read in the image and create the image data
    image = CubeFile.open(path)
    data = image.get_image_array()
    #Save to png: Image.fromarray(data[0]).save('*.png')
    #fx(f1="${cube}+2" f2="${cube}+5" f3="${cube}+6" to="${base}.LS.cub" equation='pi*0.0264*cos(f3*pi/180)/(cos(f3*pi/180)+cos(f2*pi/180)) * e^(-0.0327*f1 + 0.00022842*f1^2 + -0.0000014225*f1^3)')
    pysis.fx
    #editlab from="${base}.LS.cub" options=setkey keyword=ControlSN value=ImgSimLS grpname=Instrument
    pysis.editlab
    pysis.lowpass
